In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd
import time
import math

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#ML
from sklearn.preprocessing import LabelBinarizer
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-play-statistics-dataset-2004-to-present/players.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/draft.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/passDef.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/kickReturns.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/rusher.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/games.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/penalties.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/officials.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/passer.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/qhHits.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/interceptions.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/gameParticipation.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/combine.csv
/kaggle/input/nfl-play-statistics-dataset-2004-to-present/fumbles.csv
/ka

In [2]:
df_rusher=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/rusher.csv')
df_qhHits=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/qhHits.csv')
df_gameParticipation=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/gameParticipation.csv')
df_draft=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/draft.csv')
df_passDef=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/passDef.csv')
df_tackles=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/tackles.csv')
df_games=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/games.csv')
df_combine=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/combine.csv')
df_passer=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/passer.csv')
df_plays=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/plays.csv')
df_sacks=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/sacks.csv')
df_kicks=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/kicks.csv')
df_interceptions=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/interceptions.csv')
df_players=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/players.csv')
df_penalties=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/penalties.csv')
df_reciever=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/receiver.csv')
df_officieals=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/officials.csv')
df_fumblForced=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/fumblForced.csv')
df_kickReturns=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/kickReturns.csv')
df_fumbles=pd.read_csv('/kaggle/input/nfl-play-statistics-dataset-2004-to-present/fumbles.csv')

In [3]:
dic=df_draft[['teamId','draftTeam']].drop_duplicates()
dic=dic.set_index(['draftTeam'])

df_games_1=df_games[['gameId','season','gameDate','homeTeamId','visitorTeamId']]
df_plays_1=df_plays[['playId','gameId','possessionTeamId','distanceToGoalPre','distanceToGoalPost','down','distance']]
result = pd.merge(df_plays_1, df_games_1, how='right', on=['gameId'])

df_punt=df_kicks[(df_kicks.kickType == 'punt')]
df_fieldgoal=df_kicks[(df_kicks.kickType == 'field goal')]

df_passer['play_selection']='pass'
df_rusher['play_selection']='rush'
df_punt['play_selection']='punt'
df_fieldgoal['play_selection']='fieldgoal'

df_passer_1=df_passer[['playId','teamId','passOutcomes','passLength','play_selection']]
df_rusher_1=df_rusher[['playId','teamId','rushYards','play_selection']]
df_punter_1=df_punt[['playId','teamId','kickLength','play_selection']]
df_fieldgoal_1=df_fieldgoal[['playId','teamId','kickOutcome','kickLength','play_selection']]


result_pass = pd.merge(result, df_passer_1, how='inner', on=['playId'])
result_rush = pd.merge(result, df_rusher_1, how='inner', on=['playId'])
result_punt = pd.merge(result, df_punter_1, how='inner', on=['playId'])
result_fldg = pd.merge(result, df_fieldgoal_1, how='inner', on=['playId'])


final_df=result_pass.append(result_rush, ignore_index = True)
final_df=final_df.append(result_punt, ignore_index = True)
final_df=final_df.append(result_fldg, ignore_index = True)

final_df['kickOutcome']=final_df['kickOutcome'].replace({'missed':'missed','good':'made','blocked':'blocked'})
final_df.rename(columns={'possessionTeamId':'posteam','play_selection':'play_type','kickOutcome':'field_goal_result','distanceToGoalPre':'yardline_100', 'distance':'ydstogo', 'kickLength':'return_yards'}, inplace=True)

final_df['play_type']=final_df['play_type'].replace({'pass':'pass','rush':'run','fieldgoal':'field_goal','punt':'punt'})
final_df['yards_gained']=final_df['yardline_100']-final_df['distanceToGoalPost']

final_df.loc[final_df['play_type'] =='field_goal', 'field_goal_attempt'] = int(1)
final_df.loc[final_df['play_type'] !='field_goal', 'field_goal_attempt'] = int(0)

final_df.loc[final_df['passOutcomes'] =='complete', 'complete_pass'] = int(1)
final_df.loc[final_df['passOutcomes'] !='complete', 'complete_pass'] = int(0)

final_df.loc[final_df['passOutcomes'] =='incomplete', 'incomplete_pass'] = int(1)
final_df.loc[final_df['passOutcomes'] !='incomplete', 'incomplete_pass'] = int(0)

final_df.loc[final_df['passOutcomes'] =='interception', 'interception'] = int(1)
final_df.loc[final_df['passOutcomes'] !='interception', 'interception'] = int(0)

final_df.loc[final_df['passOutcomes'] =='sack', 'sack'] = int(1)
final_df.loc[final_df['passOutcomes'] !='sack', 'sack'] = int(0)

In [ ]:
final_df.to_csv('final_data.csv')

In [5]:
def play_choices(df_play_1):
    values = df_play_1['play_type'].value_counts(dropna=False, normalize=True).keys().tolist()
    counts = df_play_1['play_type'].value_counts(dropna=False, normalize=True).tolist()
    p = dict(zip(values, counts))
    result = rd.choices(list(p.keys()), weights=list(p.values()))[0]
    return result

In [6]:
class match:
    def __init__(self):
        self.yardline100=15
        self.time=0
        self.qrtr=1
        self.off=rd.choice(['L','V'])
        l=['L','V']
        l.remove(self.off)
        self.dfe=l[0]
        self.scnd_start=self.dfe
        self.down=1
        self.downyards=0
        self.end=False
        
    def end_match(self, l_score, v_score):
        if self.qrtr==5 and l_score!=v_score:
            self.end=True
        elif self.qrtr==6:
            self.end=True
    
    def punt(self, yard):
        self.yardline100=yard
    
    def update_qrtr(self):
        if self.time>=900:
            self.time=0
            #print('end of qrtr number ',self.qrtr)
            self.qrtr+=1
            
    
    def change_off(self):
        if self.off=='L':
            self.off='V'
        else:
            self.off='L'
    
    def update_down(self):
        self.down+=1
        
    def reset_down(self):
        self.down=1
        self.downyards=0
    
    def update_yardline100(self, yards):
        self.yardline100+=yards
        
    def update_time(self, time_2):
        self.time+=time_2

In [7]:
class team:
    def __init__(self,name,data):
        self.data=data
        self.name=name
        self.score=0
        self.passOutcomes=''
    
    def gen_distr(self):
        
        ############
        #FIELD GOAL#
        ############
        team=self.name
        columns=['posteam','yardline_100','field_goal_attempt','field_goal_result']
        df_fg=self.data[columns]
        df_fg=df_fg[df_fg.posteam==team]
        columns=['field_goal_result','field_goal_attempt','yardline_100']
        df_fg=df_fg[columns]
        data=df_fg.dropna()
        predict = 'field_goal_result'
        data=data.replace('missed', 0)
        data=data.replace('blocked', 0)
        data=data.replace('made',1)
        data=data[['field_goal_result','yardline_100']]
        X = np.array(data.drop([predict], 1))
        y = np.array(data[predict])
        x_train, y_train = X, y
        self.fg_distr = linear_model.LogisticRegression()
        self.fg_distr.fit(x_train, y_train)
        
        #############
        #PLAY SELECT#
        #############
        team=self.name
        columns=['posteam','yardline_100','down','play_type','ydstogo']
        df_play=self.data[columns]
        df_play=df_play[df_play.posteam==team]
        self.df_play=df_play[(df_play.play_type=='pass')|(df_play.play_type=='run')|(df_play.play_type=='field_goal')|(df_play.play_type=='punt')]
    
    def time_2(self,play_select,yards):
        time = abs(yards)*2.5+15
        return time
        
    def play_select(self,yardline100,down, downyards):
        ###########
        #REFERENCE####################################
        #link: https://www.mit.edu/~dimitrib/csts.pdf#
        ##############################################
        if down == 1:
            result='pass'
        
        elif down==2:
            if downyards<3:
                result='run'
            else:
                result='pass'
        
        elif down==3:
            if yardline100<41:
                if downyards<3:
                    df_play_1=self.df_play[(self.df_play.yardline_100<41)&(self.df_play.down==3)&(self.df_play.ydstogo<3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run'))]
                    result = play_choices(df_play_1)
                    
                else:
                    df_play_1=self.df_play[(self.df_play.yardline_100<41)&(self.df_play.down==3)&(self.df_play.ydstogo>=3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run'))]
                    result = play_choices(df_play_1)                 
            else:
                if downyards<3:
                    df_play_1=self.df_play[(self.df_play.yardline_100>=41)&(self.df_play.down==3)&(self.df_play.ydstogo<3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run'))]
                    result = play_choices(df_play_1)
                    
                else:
                    df_play_1=self.df_play[(self.df_play.yardline_100>=41)&(self.df_play.down==3)&(self.df_play.ydstogo>=3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run'))]
                    result = play_choices(df_play_1)                  
        else:
            if yardline100<41:
                if downyards<3:
                    df_play_1=self.df_play[(self.df_play.yardline_100<41)&(self.df_play.down==4)&(self.df_play.ydstogo<3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run')|(self.df_play.play_type=='field_goal'))]
                    result = play_choices(df_play_1)                   
                
                else:
                    df_play_1=self.df_play[(self.df_play.yardline_100<41)&(self.df_play.down==4)&(self.df_play.ydstogo>=3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run')|(self.df_play.play_type=='field_goal'))]
                    result = play_choices(df_play_1)             
            else:
                if downyards<3:
                    df_play_1=self.df_play[(self.df_play.yardline_100>=41)&(self.df_play.down==4)&(self.df_play.ydstogo<3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run')|(self.df_play.play_type=='punt'))]
                    result = play_choices(df_play_1)          
                else:
                    df_play_1=self.df_play[(self.df_play.yardline_100>=41)&(self.df_play.down==4)&(self.df_play.ydstogo>=3)&((self.df_play.play_type=='pass')|(self.df_play.play_type=='run')|(self.df_play.play_type=='punt'))]
                    result = play_choices(df_play_1)
        return result
            
    def play_result(self):
        team=self.name
        columns=['posteam','play_type','interception','complete_pass','incomplete_pass','sack']
        df_play_result=self.data[columns]
        df_play_result=df_play_result[(df_play_result.posteam==team)&(df_play_result.play_type=='pass')].dropna()
        df_play_result['interception']= df_play_result['interception'].astype(str)
        df_play_result['complete_pass']= df_play_result['complete_pass'].astype(str)
        df_play_result['incomplete_pass']= df_play_result['incomplete_pass'].astype(str)
        df_play_result['sack']= df_play_result['sack'].astype(str)
        p_int = df_play_result['interception'].value_counts(normalize=True) * 100
        p_com = df_play_result['complete_pass'].value_counts(normalize=True) * 100
        p_inc = df_play_result['incomplete_pass'].value_counts(normalize=True) * 100
        p_sck = df_play_result['sack'].value_counts(normalize=True) * 100
        result = rd.choices(['interception','complete','incomplete','sack'], weights=[p_int['1.0'], p_com['1.0'], p_inc['1.0'], p_sck['1.0']])[0]
        self.passOutcomes=result
    
    def yards(self, play_select):
        if play_select=='run':
            columns=['posteam','play_type','yards_gained']
            df_yardsgained=self.data[columns]
            df_yardsgained=df_yardsgained[(df_yardsgained.posteam==self.name) & (df_yardsgained.play_type=='run')].dropna()
            mu=df_yardsgained.yards_gained.mean()
            sigma=df_yardsgained.yards_gained.std()
            yards=int(np.random.normal(mu, sigma,1)[0])
            
        elif play_select=='pass':
            columns=['posteam','play_type','yards_gained','passOutcomes']
            df_yardsgained=self.data[columns]
            df_yardsgained=df_yardsgained[(df_yardsgained.posteam==self.name) & (df_yardsgained.play_type=='pass') & (df_yardsgained.passOutcomes==self.passOutcomes)].dropna()
            mu=df_yardsgained.yards_gained.mean()
            sigma=df_yardsgained.yards_gained.std()
            yards=int(np.random.normal(mu, sigma,1)[0])
            
        elif play_select=='punt':
            columns=['posteam','return_yards']
            df_yardsgained=self.data[columns]
            df_yardsgained=df_yardsgained[(df_yardsgained.posteam==self.name)].dropna()
            mu=df_yardsgained.return_yards.mean()
            sigma=df_yardsgained.return_yards.std()
            yards=int(np.random.normal(mu, sigma,1)[0])
            
        return yards
        
    def result_fg(self,yardline):
        distr=self.fg_distr.predict_proba(X=[[yardline]])
        result=rd.choices(('G','F'),weights=(distr[0][1],distr[0][0]))[0]
        if result=='G':
            return 1
        else:
            return 0

In [9]:
def sim_1(l, v, data):
    teams = {'L':team(l,data), 'V':team(v,data)}
    for t in teams:
        teams[t].gen_distr()
    match_ = match()
    return sim_2(match_, teams)

def sim_2(match, teams):
    match.yardline100 = teams[match.dfe].yards('punt')
    #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.dfe].name,':Punt')
    return sim_3(match, teams)

def sim_3(match, teams):
    match.down=1
    match.downyard=0
    match.end_match(teams['L'].score, teams['V'].score)
    return sim_4(match, teams)

def sim_4(match, teams):
    if match.end==True:
        #print(teams['L'].name, ':', teams['L'].score)
        #print(teams['V'].name, ':', teams['V'].score)
        #if teams['L'].score > teams['V'].score:
        #    return ('L')
        #if teams['L'].score < teams['V'].score:
        #    return('V')
        #else:
        #    return('T')

        return([teams['L'].score,teams['V'].score])
        
    
    else:
        play_select=teams[match.off].play_select(match.yardline100, match.down, match.downyards)
        if play_select=='run':
            yards=teams[match.off].yards(play_select)
            time_2=teams[match.off].time_2(play_select, yards)
            #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':Run')
            return sim_5(match, teams, time_2, yards)
        
        elif play_select == 'pass':
            teams[match.off].play_result() #pass Outcomes
            if teams[match.off].passOutcomes=='incomplete':
                time_2=10
                yards=0
                #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':Incomplete pass')
                return sim_5(match, teams, time_2, yards)
            
            else:
                yards=teams[match.off].yards(play_select)
                time_2=teams[match.off].time_2(play_select, yards)
                if teams[match.off].passOutcomes=='interception':
                    match.time+=time_2
                    match.update_qrtr()
                    #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':Intercepted pass!!!')
                    match.yardline100=100-(match.yardline100+yards)
                    match.change_off()
                    return sim_3(match, teams)
                
                else:
                    #if teams[match.off].passOutcomes=='sack':
                        #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':Sack')
                    #else:
                        #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':Completed Pass')
                    return sim_5(match, teams, time_2, yards)
                    
                
        elif play_select=='field_goal':
            result=teams[match.off].result_fg(match.yardline100)
            #if result==1:
                #print('FieldGoal=Good(',match.yardline100,')')
            #else:
                #print('FieldGoal=Fail(',match.yardline100,')')
            teams[match.off].score+=3*result
            match.change_off()
            return sim_2(match, teams)
        
        else:
            match.change_off()
            return sim_2(match, teams)

def sim_5(match, teams, time_2, yards):
    match.time+=time_2
    match.update_qrtr()
    match.yardline100+=yards
    match.downyards+=yards
    if match.yardline100>100:
        teams[match.off].score+=6
        result=teams[match.off].result_fg(10)
        teams[match.off].score+=result
        #print('(time:',match.time,')','(down:',match.down,')','(yrdline100:',match.yardline100,')',teams[match.off].name,':TOUCHDOWN!!!')
        match.change_off()
        if match.qrtr==3 and match.time==0:
            match.off=match.scnd_start
            return sim_2(match, teams)
        else:
            return sim_2(match, teams)
    
    else:
        if match.downyards > 10:
            match.down=1
            match.downyards=0
            if match.qrtr==3 and match.time==0:
                match.off=match.scnd_start
                return sim_2(match, teams)
            else:
                return sim_4(match, teams)
        
        else:
            if match.down==4:
                match.change_off()
                match.yardline100=100-match.yardline100
                if match.qrtr==3 and match.time==0:
                    match.off=match.scnd_start
                    return sim_2(match, teams)
                else:
                    return sim_3(match, teams)
            
            else:
                match.down+=1
                if match.qrtr==3 and match.time==0:
                    match.off=match.scnd_start
                    return sim_2(match, teams)
                else:
                    return sim_4(match, teams)

**Test**
* KC v/s SF (SuperBowl LIV)

In [37]:
L_name_code='SF'
V_name_code='KC'
score=[]
data_season=final_df[final_df.season ==2019]
L_name=dic.loc[L_name_code].teamId
V_name=dic.loc[V_name_code].teamId
m=sim_1(L_name,V_name,data_season)
print(L_name_code+":"+str(m[0]))
print(V_name_code+":"+str(m[1]))

SF:14
KC:49
